In [1]:
import pandas as pd
import sacrebleu
from bleurt import score
from bert_score import BERTScorer
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from rouge_score import rouge_scorer



2025-02-18 00:12:15.715965: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-18 00:12:18.383544: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-18 00:12:18.401405: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-18 00:12:27.118108: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/lianfu_umass_edu/.conda/envs/lianfu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
class EvaluateMatric:
    def __init__(self, datapath,generated,target,original):
        self.data = pd.read_json(datapath)
        self.generated = self.data[generated].tolist()
        self.target = self.data[target].tolist()
        self.source = self.data[original].tolist()
        self.res = {}
        self.path = datapath
    def create_comet_files(self):
        # Extract file base name without extension
        # print(f"Base file name: {base_name}")
        # print(f"src file path: {src_file}")
        # print(f"hyp file path: {hyp_file}")
        # print(f"ref file path: {ref_file}")

        base_name = os.path.splitext(self.path)[0]
        # Define file paths for src, hyp, and ref files
        src_file = base_name + "_src.txt"
        hyp_file = base_name + "_hyp.txt"
        ref_file = base_name + "_ref.txt"

        # Check and create files if they do not exist
        if not os.path.exists(src_file):
            print(f"{src_file} does not exist, creating it.")
            with open(src_file, 'w',encoding='utf-8') as f:
                for sentence in self.source:
                    f.write(sentence + '\n')
            print(f"{src_file} created.")

        if not os.path.exists(hyp_file):
            print(f"{hyp_file} does not exist, creating it.")
            with open(hyp_file, 'w',encoding='utf-8') as f:
                for sentence in self.generated:
                    sentence = sentence.replace('\n', '').replace('\r', '')
                    f.write(sentence + '\n')
            print(f"{hyp_file} created.")

        if not os.path.exists(ref_file):
            print(f"{ref_file} does not exist, creating it.")
            with open(ref_file, 'w',encoding='utf-8') as f:
                for sentence in self.target:
                    f.write(sentence + '\n')
            print(f"{ref_file} created.")

        return src_file, hyp_file, ref_file
    def _rouge(self):

        r1 = {'p': [], 'r' : [], 'f': []}
        r2 = {'p': [], 'r' : [], 'f': []}
        rl = {'p': [], 'r' : [], 'f': []}
        
        rouge_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

        for i in range(len(self.data)):
            all_rouge_scores = rouge_instance.score(self.target[i], self.generated[i])
            r1['p'].append(all_rouge_scores['rouge1'][0])
            r1['r'].append(all_rouge_scores['rouge1'][1])
            r1['f'].append(all_rouge_scores['rouge1'][2])

            r2['p'].append(all_rouge_scores['rouge2'][0])
            r2['r'].append(all_rouge_scores['rouge2'][1])
            r2['f'].append(all_rouge_scores['rouge2'][2])

            rl['p'].append(all_rouge_scores['rougeL'][0])
            rl['r'].append(all_rouge_scores['rougeL'][1])
            rl['f'].append(all_rouge_scores['rougeL'][2])

        self.res['rouge-1'] = (sum(r1['p'])/len(r1['p']), sum(r1['r'])/len(r1['r']), sum(r1['f'])/len(r1['f']))
        self.res['rouge-2'] = (sum(r2['p'])/len(r2['p']), sum(r2['r'])/len(r2['r']), sum(r2['f'])/len(r2['f']))
        self.res['rouge-l'] = (sum(rl['p'])/len(rl['p']), sum(rl['r'])/len(rl['r']), sum(rl['f'])/len(rl['f']))
    def bleu(self):
        src_file, hyp_file, ref_file = self.create_comet_files()
        #sacrebleu reference.tokenized -i hypothesis.tokenized --detok
        # os.system(f"sacrebleu {hyp_file} -i {ref_file} --detok")
        # bleu_scores = sacrebleu.corpus_bleu(self.generated,self.target).score
        # self.res['bleu'] = bleu_scores
        os.system(f"sacrebleu {ref_file} -i {hyp_file} -m bleu")
        # print(f"BLEU: {bleu_scores}")
    def bertscore(self):
        # bert_scorer = BERTScorer(lang="others", rescale_with_baseline=True, model_type="microsoft/deberta-xlarge-mnli")
        # bert_P, bert_R, bert_F = bert_scorer.score(self.target, self.generated)
        # self.res['bertscore_p'] = bert_P.mean().item()
        # self.res['bertscore_r'] = bert_R.mean().item()
        # self.res['bertscore_f'] = bert_F.mean().item()
        # print(f"BERTScore: P: {bert_P.mean().item()} R: {bert_R.mean().item()} F: {bert_F.mean().item()}")
        src_file, hyp_file, ref_file = self.create_comet_files()
        os.system(f"bert-score -r {ref_file} -c {hyp_file} --lang sp --rescale_with_baseline")
    def bleurt(self):
        bleurt_scorer = score.BleurtScorer(r"/project/pi_hongyu_umass_edu/zonghai/clinical-llm-alignment/durga_sandeep/Evaluation_Metrics/ClinicalBLEURT/ClinicalBLEURT")
        bleurt_scores = bleurt_scorer.score(references=self.target, candidates=self.generated)
        self.res['bleurt'] = sum(bleurt_scores)/len(bleurt_scores)
        print(f"BLEURT: {sum(bleurt_scores)/len(bleurt_scores)}")
#         src_file, hyp_file, ref_file = self.create_comet_files()
#         os.system(f"python -m bleurt.score_files \
#   -candidate_file={hyp_file} \
#   -reference_file={ref_file} \
#   -bleurt_checkpoint=/project/pi_hongyu_umass_edu/zonghai/hospital_translation/BLEURT-20")
    def rouge(self):
        self._rouge()
        print(f"ROUGE-1: {self.res['rouge-1']}")
        print(f"ROUGE-2: {self.res['rouge-2']}")
        # print(f"ROUGE-L: {self.res['rouge-l']}")
    def comet(self, model="wmt20-comet-da"):
        # Create the necessary comet files
        src_file, hyp_file, ref_file = self.create_comet_files()

        os.system(f"comet-score -s {src_file} -t {hyp_file} -r {ref_file}")
    def _llm_as_a_judge(self, judge_model='gpt-4o-mini'):
        # if 'dialogue' not in self.data.columns:
        #     print("LLM as a judge is not applicable")
        #     return 
        
        self.data['judge_prompt'] = self.data.apply(lambda x: llm_as_a_judge_prompt(x['original_english'],x['translated_spanish']), 1)

        all_responses = []
        all_scores = []
        for i in range(len(self.data)):
            
            judge_prompt = self.data.iloc[i]['judge_prompt']

            try:
                response = get_openai_response(prompt=judge_prompt, model=judge_model)
            except:
                print("########## Issue with OpenAI API")
                response = ""

            all_responses.append(response)
            
            score = get_final_score(response)
            all_scores.append(score)

        self.data['gpt_4o_mini_responses'] = all_responses
        self.data['gpt_4o_mini_scores'] = all_scores

        self.res['llm_as_a_judge'] = self.data['gpt_4o_mini_scores'].describe()['mean']
        print("LLM-as-a-Judge: ", self.res['llm_as_a_judge'])
    def evaluate(self,method):
        if method == "BLEU":
            self.bleu()
        if method == "BERTSCORE":
            self.bertscore()
        if method == "BLEURT":
            self.bleurt()
        if method == "ROUGE":
            self.rouge()
        if method == "COMET":
            self.comet()
        if method == "LLM_AS_A_JUDGE":
            self._llm_as_a_judge()
    



In [32]:

matric = EvaluateMatric("/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/DeepSeek-R1-Distill-Qwen-14B_without_finetune.json",generated="translated_spanish",target="original_spanish",original="original_english")
matric.evaluate("BLEU")

{
 "name": "BLEU",
 "score": 29.8,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.5.1",
 "verbose_score": "53.8/34.0/24.2/17.8 (BP = 1.000 ratio = 1.021 hyp_len = 3528 ref_len = 3454)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.5.1"
}


In [14]:

matric.evaluate("BLEURT")

INFO:tensorflow:Reading checkpoint /project/pi_hongyu_umass_edu/zonghai/clinical-llm-alignment/durga_sandeep/Evaluation_Metrics/ClinicalBLEURT/ClinicalBLEURT.


INFO:tensorflow:Reading checkpoint /project/pi_hongyu_umass_edu/zonghai/clinical-llm-alignment/durga_sandeep/Evaluation_Metrics/ClinicalBLEURT/ClinicalBLEURT.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint bert_custom


...

INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


BLEURT: 0.836385410875082


In [19]:
# bert_scorer = BERTScorer(lang="others", rescale_with_baseline=False, model_type="microsoft/deberta-xlarge-mnli")
matric.evaluate("BERTSCORE")

2025-02-17 22:32:56.550346: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 22:32:59.740480: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 22:33:08.479445: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.46.3)-rescaled_fast-tokenizer P: 0.799567 R: 0.837346 F1: 0.817007


In [10]:
matric.evaluate("COMET")

2025-02-17 18:28:05.250713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-17 18:28:05.842211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-17 18:28:06.022078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 18:28:07.556765: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 18:28:13.536415: W tensorflow/compiler/tf2

/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/DeepSeek-R1-Distill-Qwen-14B_without_finetune_hyp.txt	Segment 0	score: 0.7507
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/DeepSeek-R1-Distill-Qwen-14B_without_finetune_hyp.txt	Segment 1	score: 0.6891
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/DeepSeek-R1-Distill-Qwen-14B_without_finetune_hyp.txt	Segment 2	score: 0.7110
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/DeepSeek-R1-Distill-Qwen-14B_without_finetune_hyp.txt	Segment 3	score: 0.7942
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/DeepSeek-R1-Distill-Qwen-14B_without_finetune_hyp.txt	Segment 4	score: 0.7673
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/DeepSeek-R1-Distill-Qwen-14B_without_finetune_hyp.txt	Segment 5	score: 0.7934
/project/p

In [11]:
matric.evaluate("ROUGE")

ROUGE-1: (0.5492833146715158, 0.5851283728601392, 0.5488622368078425)
ROUGE-2: (0.35316686191768487, 0.37345473508064, 0.3505245523519389)


In [15]:
from gpt4o_mini_as_a_judge import llm_as_a_judge_prompt, get_openai_response, get_final_score
matric.evaluate("LLM_AS_A_JUDGE")

LLM-as-a-Judge:  -1.0


In [ ]:
matric = EvaluateMatric("/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/NLLB_direct.json",generated="translated_spanish",target="original_spanish",original="original_english")
# matric.evaluate("BLEU")
matric.evaluate("COMET")

2025-02-17 23:02:55.829300: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 23:02:58.840088: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 23:03:04.868557: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Seed set to 1
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 18267.87it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../../home/lianfu_umass_edu/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/f49d328952c3470eff6bb6f545d62bfdb6e66304/checkp

/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/NLLB_direct_hyp.txt	Segment 0	score: 0.9303
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/NLLB_direct_hyp.txt	Segment 1	score: 0.8856
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/NLLB_direct_hyp.txt	Segment 2	score: 0.8868
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/NLLB_direct_hyp.txt	Segment 3	score: 0.8682
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/NLLB_direct_hyp.txt	Segment 4	score: 0.8984
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/NLLB_direct_hyp.txt	Segment 5	score: 0.8374
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/NLLB_direct_hyp.txt	Segment 6	score: 0.8904
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/d

In [29]:
matric = EvaluateMatric(r"/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/bleu_chrf_score_DeepSeek-R1-Distill-Llama-8B.json",generated="tran_spa",target="original_spa",original="original_eng")
# matric.evaluate("BLEURT")
# matric.evaluate("BERTSCORE")
matric.evaluate("COMET")

2025-02-17 22:57:06.692682: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 22:57:11.970805: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 22:57:22.992114: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Seed set to 1
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 42799.02it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../../home/lianfu_umass_edu/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/f49d328952c3470eff6bb6f545d62bfdb6e66304/checkp

/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/bleu_chrf_score_DeepSeek-R1-Distill-Llama-8B_hyp.txt	Segment 0	score: 0.7596
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/bleu_chrf_score_DeepSeek-R1-Distill-Llama-8B_hyp.txt	Segment 1	score: 0.7491
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/bleu_chrf_score_DeepSeek-R1-Distill-Llama-8B_hyp.txt	Segment 2	score: 0.7912
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/bleu_chrf_score_DeepSeek-R1-Distill-Llama-8B_hyp.txt	Segment 3	score: 0.8376
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/bleu_chrf_score_DeepSeek-R1-Distill-Llama-8B_hyp.txt	Segment 4	score: 0.8421
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/bleu_chrf_score_DeepSeek-R1-Distill-Llama-8B_hyp.txt	Segment 5	score: 0.8225
/project/pi_hong

In [37]:
matric = EvaluateMatric("/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/phi-4_without_finetune.json",generated="translated_spanish",target="original_spanish",original="original_english")
# matric.evaluate("BLEURT")
matric.evaluate("COMET")
# matric.evaluate("ROUGE")
# matric.evaluate("BLEU")
# matric.evaluate("BERTSCORE")

2025-02-18 03:50:12.080200: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-18 03:50:14.134987: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-18 03:50:18.967165: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Seed set to 1
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 39494.39it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../../home/lianfu_umass_edu/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/f49d328952c3470eff6bb6f545d62bfdb6e6630

/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/phi-4_without_finetune_hyp.txt	Segment 0	score: 0.8100
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/phi-4_without_finetune_hyp.txt	Segment 1	score: 0.7611
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/phi-4_without_finetune_hyp.txt	Segment 2	score: 0.1751
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/phi-4_without_finetune_hyp.txt	Segment 3	score: 0.1804
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/phi-4_without_finetune_hyp.txt	Segment 4	score: 0.1724
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/phi-4_without_finetune_hyp.txt	Segment 5	score: 0.2115
/project/pi_hongyu_umass_edu/zonghai/hospital_translation/IFT_Metrics/IFT_Metrics/data/phi-4_without_finetune_hyp.txt	Segment 6	score: 0.1500
/proje